Недооцененная по сложности задача, решение которой хорошо описанно [тут](https://habr.com/ru/companies/yandex/articles/414997/). В итоге, я ориентировался на это же решение и подробнее не опишу. Возможно, кому-то будет понятнее мой код без использования классов (но вряд ли).

In [ ]:
import sys

n = int(sys.stdin.readline())
xy = []
right_n = 0
sum_right = 0
right_sse = 0
for _ in range(n):
    x,y = map(float,sys.stdin.readline().split())
    xy.append([x,y])
    sum_right += y
    right_n += 1
    b = sum_right/right_n
    right_diff = y - b
    right_sse += right_diff*(y-b)


xy.sort()

a = 0
sum_left = 0
left_n = 0
left_sse = 0

min_mse = right_sse

min_c = xy[0][0]
min_b = sum_right/right_n
min_a = 0

for i in range(len(xy)-1):
    cur_item = xy[i]
    next_item = xy[i+1]
    c = (cur_item[0]+next_item[0])/2
    sum_left += cur_item[1]
    sum_right -= cur_item[1]
    left_n += 1
    right_n -= 1
    left_diff = cur_item[1] - a
    right_diff = cur_item[1] - b
    a = sum_left/left_n
    b = sum_right/right_n
    left_sse += left_diff*(cur_item[1]-a)
    right_sse -= right_diff*(cur_item[1]-b)
    if cur_item[0] == next_item[0]:
        continue

    temp_mse = left_sse + right_sse
    if temp_mse<min_mse:
        min_mse = temp_mse
        min_a,min_b,min_c = a,b,c

print(min_a,min_b,min_c)